In [1]:
pip install python-levenshtein

     |████████████████████████████████| 51kB 4.0MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144797 sha256=dd42f6893ef9e0cfc77e09cfe729e209d9ce756c766f9e11932b391f635e3245
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


#   **Importing Libraries**

In [2]:
import librosa
import math
import numpy as np
import pdb
import string
from Levenshtein import distance
import csv, os, glob
from tensorflow import keras
import matplotlib.pyplot as plt
import librosa.display
import pickle

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling2D,Conv2D,GlobalAveragePooling2D

## **Time step taken**

In [3]:
time_step = 201

# **Functions provided in utils.py**

In [4]:
def wav2feat(wavfile):
    '''
    Input: audio wav file name
    Output: Magnitude spectrogram
    '''
    x, Fs = librosa.load(wavfile, sr=44100, mono=True) 
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)

def wavs2feat(wavfiles):
    '''
    Concatenate the audio files listed in wavfiles
    Input: list of audio wav file names
    Output: Magnitude spectrogram of concatenated wav
    '''
    x = []
    for wf in wavfiles:
        x1, Fs = librosa.load(wf, sr=44100, mono=True)
        x.append(x1)
    x = np.hstack(x)
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)

def read_csv(filename):
    id_label = {}
    with open(filename,'r') as fid:
        for line in fid: # '176787-5-0-27.wav,engine_idling\n'
            tokens = line.strip().split(',') # ['176787-5-0-27.wav', 'engine_idling']
            id_label[tokens[0]] = tokens[1]
    return id_label
    
def editDistance(gt, est):
    '''both are lists of labels
    E.g. gt is "dog_bark-street_music-engine_idling"
    E.g. est is "street_music-engine_idling"
    '''
    gttokens = gt.split('-')
    esttokens = est.split('-')
    # Map token to char
    tokenset = list(set(gttokens+esttokens)) # ['dog_bark', 'siren', 'street_music', 'engine_idling']
    token_char = {}
    for i in range(len(tokenset)):
        token_char[tokenset[i]] = string.ascii_uppercase[i]  # {'dog_bark': 'A', 'siren': 'B', 'street_music': 'C', 'engine_idling': 'D'}
    # convert gt and est to strings
    gtstr = [token_char[t] for t in gttokens]
    gtstr = ''.join(gtstr)  # 'BCA'
    eststr = [token_char[t] for t in esttokens]
    eststr = ''.join(eststr)  # 
    # Compare
    editdist = distance(gtstr, eststr) # 1
    score = 1 - editdist/len(gtstr)
    return editdist, score

def evals(gtcsv, estcsv, taskid):
    gt_id_label = read_csv(gtcsv)
    est_id_label = read_csv(estcsv)
    score = 0
    for id in est_id_label:
        if taskid==1:
            if est_id_label[id] == gt_id_label[id]:
                score += 1
        elif taskid==2:
            _, ss = editDistance(gt_id_label[id], est_id_label[id])
            score += ss
        else:
            pdb.set_trace()
            assert False, ["taskid not correct; it is", taskid]
    avgScore = score/len(est_id_label)
    return avgScore

#if __name__=="__main__":
#    wavs = ['../shared_train/audio_train/180937-7-3-27.wav']
#    wavs2feat(wavs)
#     # wavfiles = ['../shared_train/audio_train/180937-7-3-27.wav','../shared_train/audio_train/180937-7-3-27.wav']
#     # X = wavs2feat(wavfiles)
#     # eval('test_task1/labels.csv', 'test_task1/est.csv', 1)
#     editDistance("dog_bark-street_music-engine_idling",
#         "siren-street_music-engine_idling")

**Feature extraction of multiple files**

In [5]:
def read_audio_files(files):
    feat = [wav2feat(file) for file in files]
    return feat

**Add padding for making the dimension same for all the files**

In [6]:
def add_padding(X):
    data_samples = len(X)
    for i in range(data_samples):
        X[i] = np.pad(X[i],((0,0),(0,time_step-X[i].shape[1])))
    X = np.array(X)
    return X

**Function for One_hot_encoding**

In [ ]:
def one_hot_encoding(t_indices, N):
    '''
    Inputs:
        t_indices: [np.array] list of indices
        N: [int] total no. of classes
    Output:
        t_1hot: [np.array] one hot encoded vectors
    '''
    assert N>max(t_indices), (N, max(t_indices))
    ### WRITE YOUR CODE HERE - 2 MARKS
    l = len(t_indices)
    t_1hot = np.zeros((l,N),dtype = int)
    for i in range(l):
        t_1hot[i,t_indices[i]] = 1

    return t_1hot

**Splitting training data into training and validation data**

In [ ]:
def splitData(X,t,testFraction=0.1):
    """
    Split the data randomly into training and test sets
    Use numpy functions only
    Inputs:
        X: np array of shape (Nsamples, dim)
        t: np array of len Nsamples; can be one hot vectors or labels
        testFraction: (float) Nsamples_test = testFraction * Nsamples
    """


    ### WRITE YOUR CODE HERE - 3 MARKS
    np.random.seed(0)
    np.take(X,np.random.permutation(X.shape[0]),axis=0,out=X);
    np.random.seed(0)
    np.take(t,np.random.permutation(X.shape[0]),axis=0,out=t);
    N_test = int(X.shape[0]*testFraction)
    X_test, X_train = X[:N_test,:,:,:], X[N_test:,:,:,:]
    t_test, t_train = t[:N_test], t[N_test:]   

    return X_train, t_train, X_test, t_test

**Training Function**

In [ ]:
def train(X_train, t_train, X_test,t_test, Nepochs):
    '''
    Train a keras dense model for multi-event classification.
    Inputs:
        X_train: Data for training.
        Nepochs: number of epochs
    Return:
        model: keras model
    '''

    n_input = X_train.shape[0]
    
    model = keras.Sequential()
    model.add(Conv2D(filters = 16, kernel_size = 3, input_shape = (513,time_step,1), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Dropout(0.15))
    
    model.add(Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Dropout(0.15))
    
    model.add(Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Dropout(0.1))
    
    model.add(Conv2D(filters = 128, kernel_size = 3, activation = 'relu'))
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Dropout(0.1))
    model.add(GlobalAveragePooling2D())
    
    model.add(Dense(100,activation = 'relu'))
    model.add(Dropout(0.1))
    
    model.add(Dense(11, activation = 'softmax'))
    
    opt = keras.optimizers.Adam(learning_rate = 0.01)
    checkpoint_filepath = "/checkpoint2"
    checkpoint = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, verbose=1, monitor='val_accuracy',save_best_only=True,save_weights_only=True,mode='max')
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, t_train, epochs = Nepochs,batch_size = 100, validation_data=(X_test,t_test),callbacks=[checkpoint])   
      

    return model

**Hard Prediction**

In [ ]:
def predict(model, X_test):
    '''
    Predict class for unknown inputs
    Returns:
        y_pred: np array of predicted vectors.
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    y_pred = model.predict(X_test)
    n_output = y_pred.shape[1]
    y = np.argmax(y_pred,axis = 1)
    y_pred = one_hot_encoding(y,n_output)

    return y_pred

**Evaluation of CM**

In [ ]:
def evaluate(y_pred, t_test):
    '''
    Use only numpy
    Return CM: np.array of shape (No. of Events, No. of Events)
    '''
    n_test = y_pred.shape[0]

    n_classes = y_pred.shape[1]
    CM = np.zeros((n_classes,n_classes),dtype = int) 
    y, t = np.argmax(y_pred, axis = 1), t_test
    for i in range(n_test):
      CM[t[i],y[i]] = CM[t[i],y[i]] + 1

    return CM

# **Preprocessing of training data**

In [7]:
from google.colab import drive
drive.mount("/content/drive",force_remount = True)

Mounted at /content/drive


In [8]:
files = glob.glob("/content/drive/My Drive/audio_train_1ch/*.wav")
audio_files = [os.path.basename(file) for file in files]

*Saving extracted features of training data into sample.pkl*

In [ ]:
#X_train = read_audio_files(files)
#file_name = "/content/drive/My Drive/sample.pkl"
  
#open_file = open(file_name, "wb")
#pickle.dump(X_train, open_file)
#open_file.close()

*Loading features of training data from the saved file*

In [9]:
file_name = "/content/drive/My Drive/sample.pkl"
open_file = open(file_name, "rb")
X_train = pickle.load(open_file)
open_file.close()

*Partitioning train data according to our time step*

In [10]:
Number_of_partitions = [math.ceil(X_train[i].shape[1]/time_step) for i in range(len(X_train))]
train_data = []
for i in range(len(X_train)):
  for j in range(Number_of_partitions[i]):
    if (j+1)*time_step > X_train[i].shape[1]:
      train_data.append(X_train[i][:,j*time_step:X_train[i].shape[1]])
    else :
      train_data.append(X_train[i][:,j*time_step:(j+1)*time_step])
      
train_data = add_padding(train_data)

*Labelling each part of a audio file with its label*

In [11]:
Id_labels = read_csv("/content/drive/My Drive/labels_train.csv")
target_data = []
Label_to_index = {'dog_bark':0, 'air_conditioner':1, 'engine_idling':2, 'siren':3, 'class':4, 'gun_shot':5, 'jackhammer':6, 'drilling':7, 'children_playing':8, 'car_horn':9, 'street_music':10}
index_to_Label = {0:'dog_bark', 1:'air_conditioner', 2:'engine_idling', 3:'siren', 4:'class', 5:'gun_shot', 6:'jackhammer', 7:'drilling', 8:'children_playing', 9:'car_horn', 10:'street_music'}

for i in range(len(X_train)):
  for j in range(Number_of_partitions[i]):
    target_data.append(Label_to_index[Id_labels[audio_files[i]]])

target_data = np.array(target_data)  

# **Preprocessing of Test Data**

*Appending all .npy of test data into all_arrays2*

In [38]:
test_files2 = glob.glob("/content/drive/My Drive/test_task2/feats/*.npy")
test_audio_files2 = [os.path.basename(file)[0:-4] for file in test_files2]

fpath2 ="/content/drive/My Drive/test_data2.npy"
npyfilespath2 ="/content/drive/My Drive/test_task2/feats"   
os.chdir(npyfilespath2)
npfiles2 = glob.glob("/content/drive/My Drive/test_task2/feats/*.npy")
all_arrays2 = []
for i, npfile in enumerate(npfiles2):
    all_arrays2.append(np.load(os.path.join(npyfilespath2, npfile)))

*Partitioning test data according to our time step*

In [ ]:
Number_of_partitions = [math.ceil(all_arrays2[i].shape[1]/time_step) for i in range(len(all_arrays2))]

test_data2 = []
for i in range(len(all_arrays2)):
  for j in range(Number_of_partitions[i]):
    if (j+1)*time_step > all_arrays2[i].shape[1]:
      test_data2.append(all_arrays2[i][:,j*time_step:all_arrays2[i].shape[1]])
    else :
      test_data2.append(all_arrays2[i][:,j*time_step:(j+1)*time_step])

test_data2 = add_padding(test_data2)
test_data2 = test_data2.reshape(test_data2.shape[0],test_data2.shape[1],test_data2.shape[2],1)

# **Training**

In [ ]:
train_data = train_data.reshape(train_data.shape[0],train_data.shape[1],train_data.shape[2],1)
X_train, t_train, X_val, t_val = splitData(train_data,target_data,testFraction=0.1)

In [26]:
if __name__=="__main__":
  model = train(X_train, t_train, X_test,t_test, Nepochs=100)

Epoch 1/100
30/30 [==============================] - ETA: 0s - loss: 2.3975 - accuracy: 0.1400
Epoch 00001: val_accuracy improved from -inf to 0.21148, saving model to /checkpoint2
30/30 [==============================] - 10s 348ms/step - loss: 2.3975 - accuracy: 0.1400 - val_loss: 2.1662 - val_accuracy: 0.2115
Epoch 2/100
30/30 [==============================] - ETA: 0s - loss: 2.0407 - accuracy: 0.2583
Epoch 00002: val_accuracy improved from 0.21148 to 0.33535, saving model to /checkpoint2
30/30 [==============================] - 10s 337ms/step - loss: 2.0407 - accuracy: 0.2583 - val_loss: 1.8725 - val_accuracy: 0.3353
Epoch 3/100
30/30 [==============================] - ETA: 0s - loss: 1.7559 - accuracy: 0.3682
Epoch 00003: val_accuracy improved from 0.33535 to 0.48338, saving model to /checkpoint2
30/30 [==============================] - 10s 337ms/step - loss: 1.7559 - accuracy: 0.3682 - val_loss: 1.6046 - val_accuracy: 0.4834
Epoch 4/100
30/30 [==============================] - ET

**Test on Validation Dataset**

In [37]:
model.load_weights("/checkpoint2")
model.evaluate(X_val, t_val)
y = predict(model,X_val)
print(evaluate(y,t_val))

11/11 [==============================] - 0s 40ms/step - loss: 0.0864 - accuracy: 0.9637
[[40  0  0  0  0  0  0  0  1  0  0]
 [ 0 39  3  0  0  0  0  0  0  0  0]
 [ 0  0 22  0  0  0  0  0  0  0  0]
 [ 0  0  0 35  0  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  8  0  0  0  0  0]
 [ 0  0  0  0  0  0 43  0  0  0  0]
 [ 0  1  0  0  0  0  0 46  0  0  0]
 [ 1  0  0  0  0  1  0  0 36  0  0]
 [ 0  0  0  0  0  0  0  0  0 11  0]
 [ 0  0  0  0  0  0  0  2  1  0 39]]


# **Prediction on test data**

In [39]:
y_pred2 = predict(model, test_data2)
y_pred2 = np.argmax(y_pred2,axis = 1)

estimate = []
cur = 0

for i in range(len(all_arrays2)):
  estimate.append("")
  for j in range(Number_of_partitions[i]):
    if j==0:
      estimate[len(estimate)-1] = estimate[len(estimate)-1] + index_to_Label[y_pred2[cur]]
    else :
      if index_to_Label[y_pred2[cur-1]] != index_to_Label[y_pred2[cur]]:
        estimate[len(estimate)-1] = estimate[len(estimate)-1] + "-" + index_to_Label[y_pred2[cur]]
    cur = cur + 1

# **Save keys to csv file**

In [41]:
prediction_dict = dict(zip(test_audio_files2, estimate)) 


filename = '/content/drive/MyDrive/task2.csv'
with open(filename, 'w') as f:
    for key in prediction_dict.keys():
        f.write("%s,%s\n" % (key, prediction_dict[key]))